In [33]:
!pip install qiskit torch numpy pandas


[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: python3.10 -m pip install --upgrade pip


In [ ]:
# Library imports
%matplotlib inline
import math
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import pennylane as qml
from IPython import display
import progressbar
from PIL import Image

# Pytorch imports
import torch
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt

# Set the random seed for reproducibility
seed = 42
torch.manual_seed(seed)
np.random.seed(seed)
random.seed(seed)
device = torch.device('cpu')

In [ ]:
#Prep Training Data (Breast Cancer Wisconsin)

#Load the data file
df = pd.read_csv("data/wdbc.data", header=None)


#Assign column names
columns = ['id', 'diagnosis'] + [f'feature_{i}' for i in range(1, 31)]
df.columns = columns
#Drop the ID column
df = df.drop(columns=['id'])

#Encode diagnosis: M = 1, B = 0
df['diagnosis'] = df['diagnosis'].map({'M': 1, 'B': 0})

#Convert to numpy arrays
X = df.drop(columns=['diagnosis']).values.astype(np.float32)
Y = df['diagnosis'].values.astype(np.float32).reshape(-1, 1)

#Normalize features manually (z-score)
mean = X.mean(axis=0)
std = X.std(axis=0)
X = (X - mean) / std

#Convert to torch tensors
X_tensor = torch.tensor(X, dtype=torch.float32)
Y_tensor = torch.tensor(Y, dtype=torch.float32)

#Manual train/test split (80/20)
num_samples = X_tensor.shape[0]
indices = torch.randperm(num_samples)

split_idx = int(num_samples * 0.8)
train_indices = indices[:split_idx]
test_indices = indices[split_idx:]
X_train = X_tensor[train_indices]
Y_train = Y_tensor[train_indices]
X_test = X_tensor[test_indices]
Y_test = Y_tensor[test_indices]

#Idea: build a Quantum CNN (since this is a classification task), facilitating non-linear feed-forward parts with QSVT
#Architecture: Classical Dense (30 -> 8) -> tanh to map data to [-1,1]
                
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import TensorDataset, DataLoader

# ---------------------------------------
# 1.  Data-set and data-loader
# ---------------------------------------
batch_size = 1

train_loader = DataLoader(
    TensorDataset(X_train, Y_train), batch_size=batch_size, shuffle=True
)
test_loader = DataLoader(
    TensorDataset(X_test, Y_test), batch_size=batch_size, shuffle=False
)

# ---------------------------------------
# 2.  A tiny 1-D CNN for 30 features
# ---------------------------------------
class BreastCancerCNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv1d(in_channels=1,  out_channels=16, kernel_size=3, padding=1)
        self.bn1   = nn.BatchNorm1d(16)
        self.conv2 = nn.Conv1d(16, 32, kernel_size=3, padding=1)
        self.bn2   = nn.BatchNorm1d(32)
        self.pool  = nn.MaxPool1d(2)
        self.fc1   = nn.Linear(32 * 7, 64)   # 30 → pool → 15 → conv → pool → 7
        self.fc2   = nn.Linear(64, 1)

    def forward(self, x):
        x = x.unsqueeze(1)                  # (B, 1, 30)
        x = self.pool(F.relu(self.bn1(self.conv1(x))))  # (B,16,15)
        x = self.pool(F.relu(self.bn2(self.conv2(x))))  # (B,32,7)
        x = x.flatten(1)                                    # (B,32*7)
        x = F.relu(self.fc1(x))
        return torch.sigmoid(self.fc2(x))                   # (B,1)

diagnosis_model = BreastCancerCNN()

# ---------------------------------------
# 3.  Loss, optimiser
# ---------------------------------------
loss_fn   = nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.0002)

# ---------------------------------------
# 4.  Training loop (mini-batch SGD)
# ---------------------------------------
n_epochs = 25
for epoch in range(n_epochs):
    diagnosis_model.train()
    running_loss, correct = 0.0, 0

    for xb, yb in train_loader:
        optimizer.zero_grad()
        preds = diagnosis_model(xb)
        loss  = loss_fn(preds, yb)
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * xb.size(0)
        correct      += ((preds > 0.5).float() == yb).sum().item()

    epoch_loss = running_loss / len(train_loader.dataset)
    epoch_acc  = correct      / len(train_loader.dataset) * 100
    print(f"Epoch {epoch+1:02d} │ Train loss {epoch_loss:.4f} │ Train acc {epoch_acc:.2f}%")

# ---------------------------------------
# 5.  Testing / evaluation loop
# ---------------------------------------
diagnosis_model.eval()
test_loss, correct = 0.0, 0

with torch.no_grad():
    for xb, yb in test_loader:
        preds = diagnosis_model(xb)
        test_loss += loss_fn(preds, yb).item() * xb.size(0)
        correct   += ((preds > 0.5).float() == yb).sum().item()

test_loss /= len(test_loader.dataset)
test_acc  = correct / len(test_loader.dataset) * 100
print(f"\nTest loss {test_loss:.4f} │ Test accuracy {test_acc:.2f}%")


Epoch 01 │ Train loss 0.4450 │ Train acc 80.00%
Epoch 02 │ Train loss 0.2340 │ Train acc 91.43%
Epoch 03 │ Train loss 0.1575 │ Train acc 94.29%
Epoch 04 │ Train loss 0.1168 │ Train acc 96.04%
Epoch 05 │ Train loss 0.0912 │ Train acc 96.26%
Epoch 06 │ Train loss 0.0667 │ Train acc 97.58%
Epoch 07 │ Train loss 0.0499 │ Train acc 98.46%
Epoch 08 │ Train loss 0.0355 │ Train acc 99.56%
Epoch 09 │ Train loss 0.0297 │ Train acc 99.78%
Epoch 10 │ Train loss 0.0204 │ Train acc 100.00%
Epoch 11 │ Train loss 0.0138 │ Train acc 100.00%
Epoch 12 │ Train loss 0.0106 │ Train acc 100.00%
Epoch 13 │ Train loss 0.0080 │ Train acc 100.00%
Epoch 14 │ Train loss 0.0063 │ Train acc 100.00%
Epoch 15 │ Train loss 0.0043 │ Train acc 100.00%
Epoch 16 │ Train loss 0.0030 │ Train acc 100.00%
Epoch 17 │ Train loss 0.0033 │ Train acc 100.00%
Epoch 18 │ Train loss 0.0017 │ Train acc 100.00%
Epoch 19 │ Train loss 0.0012 │ Train acc 100.00%
Epoch 20 │ Train loss 0.0009 │ Train acc 100.00%
Epoch 21 │ Train loss 0.0008 

In [ ]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)
count_parameters(diagnosis_model)

16193

In [ ]:
from sklearn.metrics import precision_score, recall_score, f1_score

# Ensure labels are 1D integer tensors
Y_test = Y_test.view(-1).long()   # [N,1] → [N]

diagnosis_model.eval()

y_trues = []
y_preds = []

with torch.no_grad():
    for i in range(X_test.size(0)):
        x = X_test[i].to(device)           # single sample: [features]
        y_true = Y_test[i].item()          # scalar 0/1
        
        # forward expects a batch, so add a batch dim:
        out = diagnosis_model(x.unsqueeze(0))        # → shape [1, 1] or [1,2]
        
        # if you have a single-logit head:
        if out.dim()==2 and out.shape[1]==1:
            prob = out.item()
            pred = 1 if prob >= 0.5 else 0
        else:
            # two-logit head
            pred = out.argmax(dim=1).item()
        
        y_trues.append(y_true)
        y_preds.append(pred)

# now compute metrics
precision = precision_score(y_trues, y_preds)
recall    = recall_score(y_trues, y_preds)
f1        = f1_score(y_trues, y_preds)

print(f"Precision: {precision:.4f}")
print(f"Recall:    {recall:.4f}")
print(f"F1-score:  {f1:.4f}")

Precision: 0.8636
Recall:    0.9048
F1-score:  0.8837


In [ ]:
import torch
import torch.nn.functional as F
from torch.utils.data import TensorDataset, DataLoader
from sklearn.metrics import accuracy_score

def fgsm_attack(model, x, y, epsilon, device):
    x_adv = x.clone().detach().to(device).requires_grad_(True)
    out = model(x_adv)


    loss = F.binary_cross_entropy_with_logits(out, y.unsqueeze(0).float().to(device))
    model.zero_grad()
    loss.backward()
    
    x_adv = x_adv + epsilon * x_adv.grad.sign()
    return x_adv.detach()

def evaluate_robustness(model, device, X_test, Y_test, epsilons=[0.0,0.01,0.05,0.1,0.2]):
    model.eval()
    results = {}
    ds = TensorDataset(X_test, Y_test.view(-1))
    loader = DataLoader(ds, batch_size=1, shuffle=False)
    
    for eps in epsilons:
        preds = []
        trues = []
        for x, y in loader:
            x, y = x.to(device), y.to(device)
            
            if eps == 0.0:
                x_adv = x
            else:
                x_adv = fgsm_attack(model, x, y, eps, device)
            
            with torch.no_grad():
                out = model(x_adv)
                if out.dim()==2 and out.size(1)==1:
                    out = out.view(-1)
                prob = out   # [1]
                pred = (prob >= 0.5).long().item()
            
            preds.append(pred)
            trues.append(y.item())
        
        results[eps] = accuracy_score(trues, preds)
    return results
evaluate_robustness(diagnosis_model, torch.device('cpu'), X_test, Y_test)

{0.0: 0.9122807017543859,
 0.01: 0.9035087719298246,
 0.05: 0.7631578947368421,
 0.1: 0.6140350877192983,
 0.2: 0.41228070175438597}

In [ ]:
#Prep Training Data (Breast Cancer Wisconsin)

#Prep Training Data (Breast Cancer Wisconsin)

#Load the data file
df = pd.read_csv("data/winequality-red.csv", sep = ';', header=0)

Y = (df['quality'] >= 6).astype(int).values.reshape(-1,1)
X = df.drop(['quality'], axis=1).values


mean = X.mean(axis=0)
std = X.std(axis=0)
X = (X - mean) / std

#Convert to torch tensors
X_tensor = torch.tensor(X, dtype=torch.float32)
Y_tensor = torch.tensor(Y, dtype=torch.float32)
y = Y_tensor.view(-1).long()     # flatten to shape [N] and cast to int
counts = torch.bincount(y)       # bincount over 0,1
print(f"Zeros (class 0): {counts[0].item()}")
print(f"Ones  (class 1): {counts[1].item()}")

#Manual train/test split (80/20)
num_samples = X_tensor.shape[0]
indices = torch.randperm(num_samples)

split_idx = int(num_samples * 0.8)
train_indices = indices[:split_idx]
test_indices = indices[split_idx:]
X_train = X_tensor[train_indices]
Y_train = Y_tensor[train_indices]
X_test = X_tensor[test_indices]
Y_test = Y_tensor[test_indices]
print(X.shape, Y.shape)

#Idea: build a Quantum CNN (since this is a classification task), facilitating non-linear feed-forward parts with QSVT
#Architecture: Classical Dense (30 -> 8) -> tanh to map data to [-1,1]
                
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import TensorDataset, DataLoader

# ---------------------------------------
# 1.  Data-set and data-loader
# ---------------------------------------
batch_size = 1

train_loader = DataLoader(
    TensorDataset(X_train, Y_train), batch_size=batch_size, shuffle=True
)
test_loader = DataLoader(
    TensorDataset(X_test, Y_test), batch_size=batch_size, shuffle=False
)

# ---------------------------------------
# 2.  A tiny 1-D CNN for 30 features
# ---------------------------------------
class WineCNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv1d(in_channels=1,  out_channels=16, kernel_size=3, padding=1)
        self.bn1   = nn.BatchNorm1d(16)
        self.conv2 = nn.Conv1d(16, 32, kernel_size=3, padding=1)
        self.bn2   = nn.BatchNorm1d(32)
        self.pool  = nn.MaxPool1d(2)
        self.fc1   = nn.Linear(32 * 2, 32)   # 30 → pool → 15 → conv → pool → 7
        self.fc2   = nn.Linear(32, 1)

    def forward(self, x):
        x = x.unsqueeze(1)                  # (B, 1, 30)
        x = self.pool(F.relu(self.bn1(self.conv1(x))))  # (B,16,15)
        x = self.pool(F.relu(self.bn2(self.conv2(x))))  # (B,32,7)
        x = x.flatten(1)                                    # (B,32*7)
        x = F.relu(self.fc1(x))
        return torch.sigmoid(self.fc2(x))                   # (B,1)

wine_model = WineCNN()

# ---------------------------------------
# 3.  Loss, optimiser
# ---------------------------------------
loss_fn   = nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.0002)

# ---------------------------------------
# 4.  Training loop (mini-batch SGD)
# ---------------------------------------
n_epochs = 30
for epoch in range(n_epochs):
    wine_model.train()
    running_loss, correct = 0.0, 0

    for xb, yb in train_loader:
        optimizer.zero_grad()
        preds = wine_model(xb)
        loss  = loss_fn(preds, yb)
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * xb.size(0)
        correct      += ((preds > 0.5).float() == yb).sum().item()

    epoch_loss = running_loss / len(train_loader.dataset)
    epoch_acc  = correct      / len(train_loader.dataset) * 100
    print(f"Epoch {epoch+1:02d} │ Train loss {epoch_loss:.4f} │ Train acc {epoch_acc:.2f}%")

# ---------------------------------------
# 5.  Testing / evaluation loop
# ---------------------------------------
wine_model.eval()
test_loss, correct = 0.0, 0

with torch.no_grad():
    for xb, yb in test_loader:
        preds = wine_model(xb)
        test_loss += loss_fn(preds, yb).item() * xb.size(0)
        correct   += ((preds > 0.5).float() == yb).sum().item()

test_loss /= len(test_loader.dataset)
test_acc  = correct / len(test_loader.dataset) * 100
print(f"\nTest loss {test_loss:.4f} │ Test accuracy {test_acc:.2f}%")

Zeros (class 0): 744
Ones  (class 1): 855
(1599, 11) (1599, 1)
Epoch 01 │ Train loss 0.6202 │ Train acc 65.68%
Epoch 02 │ Train loss 0.5418 │ Train acc 73.96%
Epoch 03 │ Train loss 0.5113 │ Train acc 75.14%
Epoch 04 │ Train loss 0.4933 │ Train acc 76.54%
Epoch 05 │ Train loss 0.4747 │ Train acc 76.78%
Epoch 06 │ Train loss 0.4614 │ Train acc 78.81%
Epoch 07 │ Train loss 0.4485 │ Train acc 79.75%
Epoch 08 │ Train loss 0.4359 │ Train acc 80.38%
Epoch 09 │ Train loss 0.4247 │ Train acc 80.22%
Epoch 10 │ Train loss 0.4120 │ Train acc 82.10%
Epoch 11 │ Train loss 0.4004 │ Train acc 81.70%
Epoch 12 │ Train loss 0.3955 │ Train acc 82.72%
Epoch 13 │ Train loss 0.3834 │ Train acc 83.11%
Epoch 14 │ Train loss 0.3765 │ Train acc 83.97%
Epoch 15 │ Train loss 0.3669 │ Train acc 84.99%
Epoch 16 │ Train loss 0.3534 │ Train acc 85.07%
Epoch 17 │ Train loss 0.3492 │ Train acc 85.07%
Epoch 18 │ Train loss 0.3409 │ Train acc 85.46%
Epoch 19 │ Train loss 0.3318 │ Train acc 86.00%
Epoch 20 │ Train loss 0.3

In [ ]:
from sklearn.metrics import precision_score, recall_score, f1_score

# Ensure labels are 1D integer tensors
Y_test = Y_test.view(-1).long()   # [N,1] → [N]

wine_model.eval()

y_trues = []
y_preds = []

with torch.no_grad():
    for i in range(X_test.size(0)):
        x = X_test[i].to(device)           # single sample: [features]
        y_true = Y_test[i].item()          # scalar 0/1
        
        # forward expects a batch, so add a batch dim:
        out = wine_model(x.unsqueeze(0))        # → shape [1, 1] or [1,2]
        
        # if you have a single-logit head:
        if out.dim()==2 and out.shape[1]==1:
            prob = out.item()
            pred = 1 if prob >= 0.5 else 0
        else:
            # two-logit head
            pred = out.argmax(dim=1).item()
        
        y_trues.append(y_true)
        y_preds.append(pred)

# now compute metrics
precision = precision_score(y_trues, y_preds)
recall    = recall_score(y_trues, y_preds)
f1        = f1_score(y_trues, y_preds)

print(f"Precision: {precision:.4f}")
print(f"Recall:    {recall:.4f}")
print(f"F1-score:  {f1:.4f}")

Precision: 0.6761
Recall:    0.7000
F1-score:  0.6879


In [ ]:
evaluate_robustness(wine_model, torch.device('cpu'), X_test, Y_test)

{0.0: 0.6625, 0.01: 0.640625, 0.05: 0.54375, 0.1: 0.446875, 0.2: 0.296875}

In [ ]:
count_parameters(wine_model)

3841